In [1]:
import pandas as pd;
import numpy as  np;
from scipy.optimize import minimize;
import random;

In [ ]:
ratings_large = np.genfromtxt('ml-latest//ratings.csv',delimiter=',',skip_header =1)[:,0:3]

In [ ]:
ratings_small = np.genfromtxt('ml-latest-small//ratings.csv',delimiter=',',skip_header =1)[:,0:3]

In [3]:
def get_rhat(u,v):
    return np.matmul(u,np.transpose(v))

In [87]:
#min J= 1/2||R - Utranspose(V)||**2
###Calculate reconstruction error

def cal_error(r,rhat):
    return 0.5 * np.nansum(np.square(r - rhat))    

In [98]:
def matrix_factorization(ratings,alpha,beta,steps,concept=3):
    r = np.array(pd.DataFrame(ratings).pivot(0,1,2))  
    #r=ratings
    row = (r.shape[0])
    col = (r.shape[1])
    
    np.random.seed(1234)
    u = np.random.rand(row,concept)
    v = np.random.rand(col,concept)
    
    e = r - get_rhat(u,v)
    e = np.array(pd.DataFrame(e).fillna(0))
    #e1 = np.zeros((row,col))
    #
    #for i in range(row):
    #    for j in range(col):
    #        if r[i][j] > 0:
    #            e1[i,j] = r[i][j] - get_rhat(u,v)[i,j]
    #           
               
    #e = r - get_rhat(u,v)
    #print(e)
    #print(e1)
    for s in range(steps):
        u_c = u + alpha * (np.matmul(e,v) - beta * u)
        v_c = v + alpha * (np.matmul(e.T,u) - beta * v)
       
        u = np.copy(u_c)
        v = np.copy(v_c)
        e = r - get_rhat(u,v)
        np.nan_to_num(e,copy=False)
        #e = np.array(pd.DataFrame(e).fillna(0))
        if s%5 ==0:
            print(cal_error(r,get_rhat(u,v)))
    return get_rhat(u,v)

In [104]:
matrix_factorization(ratings_small[1:10,],0.0002,0.02,500)

65.66309563717981
64.41749176546698
63.18416452439069
61.96284654976364
60.75330909807426
59.5553598902297
58.36884102710042
57.193626969366804
56.02962257511298
54.876761189492235
53.73500278160613
52.60433212450063
51.48475701489138
50.37630652988825
49.279029318600934
48.1929919270725
47.11827715550921
46.0549824472542
45.003218309388366
43.963106765238244
42.9347798394249
41.918378076402846
40.914049093712684
39.92194617140697
38.942226879305124
37.97505174389181
37.020582956793724
36.078983126852954
35.15041407786343
34.235035694048676
33.333004815339
32.44447418445237
31.56959144769972
30.70849821132343
29.861329155037936
29.02821120427889
28.209262762481124
27.40459300450123
26.61430123207836
25.838476291991178
25.07719605732119
24.330526971977303
23.598523658374468
22.881228587895393
22.178671813499754
21.490870763583644
20.81783009593595
20.159541610389496
19.515984218526498
18.88712396857168
18.27291412339417
17.67329528934358
17.088195593466544
16.517530906490357
15.96120510

array([[3.77018608, 3.83579364, 4.02584207, 4.10757939, 2.53525987,
        4.39718807, 2.85557176, 4.33599078, 3.94134842]])

In [52]:
ratings_small.shape

#np.array(pd.DataFrame(ratings_small[1:5,:]).pivot(0,1,2)) 

(100836, 3)